# 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic')

# 모델링
from sklearn.model_selection import train_test_split
import xgboost as XGB
from xgboost import XGBRegressor
from xgboost import plot_importance
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
import sklearn
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from catboost import CatBoostRegressor,Pool
from sklearn.metrics import mean_squared_error 

# 데이터 불러오기

In [ ]:
preprocessed_data = pd.read_pickle('/home/aiffeldefinition/해커톤/data/대여시간구간화_60.pkl')
preprocessed_data.head()

In [ ]:
preprocessed_df = preprocessed_data.copy()

In [ ]:
def 컬럼6_classification(df):
  A_ratio = round(len(df[df['컬럼6']=='A']) / (len(df[df['컬럼6']=='B']) + len(df[df['컬럼6']=='A'])), 3)
  B_ratio = 1 - A_ratio
  unknown = df[df['컬럼6']=='unknown']

  # 비율에 맞게 샘플링
  na_to_B = unknown.sample(frac=B_ratio, random_state=302)
  B_index = na_to_B.index
  A_index = [x for x in unknown.index if x not in B_index]

  # 입력
  for i in B_index:  
    df['컬럼6'][i] = 'B'

  for j in A_index:
    df['컬럼6'][j] = 'A'

In [ ]:
컬럼6_classification(preprocessed_df)

In [ ]:
preprocessed_df['MP'] = (preprocessed_df['컬럼8'] + preprocessed_df['컬럼9']) / 2

In [ ]:
cluster_01 = preprocessed_df[(preprocessed_df['컬럼2'] == 'AAA') | (preprocessed_df['컬럼2'] == 'AAB')]
cluster_02 = preprocessed_df[(preprocessed_df['컬럼2'] == 'AAC')]
cluster_1 = preprocessed_df[(preprocessed_df['컬럼2'] == 'ABA') | (preprocessed_df['컬럼2'] == 'ABB') | (preprocessed_df['컬럼2'] == 'ABC') | (preprocessed_df['컬럼2'] == 'ABD') | (preprocessed_df['컬럼2'] == 'ABE')]
cluster_2 = preprocessed_df[(preprocessed_df['컬럼2'] == 'ACA') | (preprocessed_df['컬럼2'] == 'ACB') | (preprocessed_df['컬럼2'] == 'ACC') | (preprocessed_df['컬럼2'] == 'ACD') | (preprocessed_df['컬럼2'] == 'ACE')]
cluster_3 = preprocessed_df[(preprocessed_df['컬럼2'] == 'ADA') | (preprocessed_df['컬럼2'] == 'ADB') | (preprocessed_df['컬럼2'] == 'ADC')]

# 개미와 베짱이 군집- 개미 Modeling


In [ ]:
cluster_01_df = cluster_01.copy()

## feature 그룹화

In [ ]:
컬럼7_list = list(cluster_01_df[(cluster_01_df['컬럼7'] == 'CA') | (cluster_01_df['컬럼7'] == 'CL')].index)
for i in 컬럼7_list:
  cluster_01_df.loc[i,'컬럼7그룹'] = 5

컬럼7_list = list(cluster_01_df[(cluster_01_df['컬럼7'] == 'CD') | (cluster_01_df['컬럼7'] == 'CK')].index)
for i in 컬럼7_list:
  cluster_01_df.loc[i,'컬럼7그룹'] = 4

컬럼7_list = list(cluster_01_df[(cluster_01_df['컬럼7'] == 'CF') | (cluster_01_df['컬럼7'] == 'CG')].index)
for i in 컬럼7_list:
  cluster_01_df.loc[i,'컬럼7그룹'] = 3

컬럼7_list = list(cluster_01_df[(cluster_01_df['컬럼7'] == 'CI') | (cluster_01_df['컬럼7'] == 'CC')].index)
for i in 컬럼7_list:
  cluster_01_df.loc[i,'컬럼7그룹'] = 2

컬럼7_list = list(cluster_01_df[(cluster_01_df['컬럼7'] == 'CJ') | (cluster_01_df['컬럼7'] == 'CE')].index)
for i in 컬럼7_list:
  cluster_01_df.loc[i,'컬럼7그룹'] = 1

In [ ]:
cluster_01_df.컬럼7그룹.value_counts()

In [ ]:
컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 12) ].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 6

컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 7) ].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 5

컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 11) | (cluster_01_df['컬럼10'] == 8) |  (cluster_01_df['컬럼10'] == 10) |  (cluster_01_df['컬럼10'] == 6)].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 4

컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 9) | (cluster_01_df['컬럼10'] == 5)].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 3

컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 4)].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 2

컬럼10_list = list(cluster_01_df[(cluster_01_df['컬럼10'] == 3) | (cluster_01_df['컬럼10'] == 1) | (cluster_01_df['컬럼10'] == 2)].index)
for i in 컬럼10_list:
  cluster_01_df.loc[i,'컬럼10_'] = 1

In [ ]:
cluster_01_df.컬럼10_.value_counts()

In [ ]:
컬럼12_list = list(cluster_01_df[(cluster_01_df['컬럼12'] == 4) | (cluster_01_df['컬럼12'] == 5)].index)
for i in 컬럼12_list:
  cluster_01_df.loc[i,'컬럼12_'] = 3

컬럼12_list = list(cluster_01_df[(cluster_01_df['컬럼12'] == 2) | (cluster_01_df['컬럼12'] == 0) |(cluster_01_df['컬럼12'] == 3)].index)
for i in 컬럼12_list:
  cluster_01_df.loc[i,'컬럼12_'] = 2

컬럼12_list = list(cluster_01_df[(cluster_01_df['컬럼12'] == 6) | (cluster_01_df['컬럼12'] == 1) ].index)
for i in 컬럼12_list:
  cluster_01_df.loc[i,'컬럼12_'] = 1

In [ ]:
cluster_01_df.컬럼12_.value_counts()

In [ ]:
컬럼2_list = list(cluster_01_df[(cluster_01_df['컬럼2'] == 'AAA')].index)
for i in 컬럼2_list:
  cluster_01_df.loc[i,'컬럼2분류'] = 2

컬럼2_list = list(cluster_01_df[(cluster_01_df['컬럼2'] == 'AAB')].index)
for i in 컬럼2_list:
  cluster_01_df.loc[i,'컬럼2분류'] = 1

In [ ]:
cluster_01_df.컬럼2분류.value_counts()

In [ ]:
컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 7) | (cluster_01_df['컬럼13'] == 6) ].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 9

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 5) | (cluster_01_df['컬럼13'] == 9) | (cluster_01_df['컬럼13'] == 8) ].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 8

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 10) ].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 7

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 18) | (cluster_01_df['컬럼13'] == 12) |  (cluster_01_df['컬럼13'] == 11)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 6

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 15) | (cluster_01_df['컬럼13'] == 14) | (cluster_01_df['컬럼13'] == 13) | (cluster_01_df['컬럼13'] == 19)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 5

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 16) | (cluster_01_df['컬럼13'] == 17)| (cluster_01_df['컬럼13'] == 4)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 4

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 21) | (cluster_01_df['컬럼13'] == 20)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 3

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 23) | (cluster_01_df['컬럼13'] == 22) | (cluster_01_df['컬럼13'] == 3)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 2

컬럼13_list = list(cluster_01_df[(cluster_01_df['컬럼13'] == 1) | (cluster_01_df['컬럼13'] == 2) | (cluster_01_df['컬럼13'] == 0)].index)
for i in 컬럼13_list:
  cluster_01_df.loc[i,'컬럼13_'] = 1

In [ ]:
cluster_01_df.컬럼13_.value_counts()

## Modeling

In [ ]:
cluster_01_df.info()

In [ ]:
cluster_01_ols = cluster_01_df.copy()
cluster_01_cat = cluster_01_df.copy()

In [ ]:
from scipy import stats

fig, axs = plt.subplots(1,3, figsize = (15,6))

sns.histplot(cluster_01_ols["컬럼15"], ax =axs[0], kde=True)
stats.probplot(cluster_01_ols["컬럼15"], dist='norm', fit=True, plot=axs[1])
sns.histplot(np.log1p(cluster_01_ols["컬럼15"]), ax=axs[2], kde=True)

axs[0].set_title("Original 컬럼15 Histogram")
axs[2].set_title("Log Transformed 컬럼15 Histogram")

plt.show()

## 개미 OLS base

In [ ]:
cluster_01_ols_base = cluster_01_ols.copy()

In [ ]:
cluster_01_ols_base.info()

In [ ]:
# 피처 제거
c_01_base_drop_cols = ['컬럼16',  '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼13_', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_01_ols_base.drop(c_01_base_drop_cols, axis=1, inplace=True)


org_SalePrice = cluster_01_ols_base["컬럼15"]
cluster_01_ols_base["컬럼15"] = np.log1p(cluster_01_ols_base["컬럼15"])


cluster_01_ols_base['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_01_ols_base.info()

In [ ]:
target_01_ols_base = cluster_01_ols_base.컬럼15
data_01_ols_base = cluster_01_ols_base.drop(columns = '컬럼15')
data_01_ols_base = sm.add_constant(data_01_ols_base)

ols_model_01_base = sm.OLS(target_01_ols_base, data_01_ols_base)
ols_result_01_base = ols_model_01_base.fit()
print(ols_result_01_base.summary())

### VIF 지수 확인

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_01_ols_base.columns),
                             cluster_01_ols_base, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
vif

In [ ]:
c_01_lr_del_vif = []

for idx, row in vif.iterrows():
  if row['VIF Factor'] >= 10:
    c_01_lr_del_vif.append(row['features'])

In [ ]:
c_01_lr_del_vif

### 기본 성능

In [ ]:
from sklearn.metrics import mean_squared_error 

target_01_ols_base = cluster_01_ols_base.컬럼15
data_01_ols_base = cluster_01_ols_base.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_01_ols_base, target_01_ols_base, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, pred_lr)

In [ ]:
def adj_r2_score(y_true, y_pred, p=x_train.shape[1]):
    return 1-(1-r2_score(y_true, y_pred)) * (len(y_true)-1) / (len(y_true) - p - 1)

adj_r2_score(y_test, pred_lr)

## 개미 OLS - Feature Engineering 

In [ ]:
cluster_01_ols_fe = cluster_01_ols.copy()
cluster_01_ols_fe.info()

### 피쳐 추가

- 평일 1구간

In [ ]:
# 평일 1구간(7-10시)
cluster_01_ols_fe['평일_1구간'] = 0

컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17'] == 0) & 
                                         (cluster_01_ols_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_01_ols_fe.loc[i,'평일_1구간'] = 1

- 평일 4구간

In [ ]:
# 평일 4구간(17-20시)
cluster_01_ols_fe['평일_4구간'] = 0

컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17']==0) & 
                                         (cluster_01_ols_fe['컬럼13'].between(17,19))].index)
for i in 컬럼13_list:
  preprocessed_df.loc[i,'평일_4구간'] = 1

- 평일 5구간

In [ ]:
# 평일 5구간(20-23시)
cluster_01_ols_fe['평일_5구간'] = 0

컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17']==0) & 
                                         (cluster_01_ols_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_01_ols_fe.loc[i,'평일_5구간'] = 1

- 주말 1구간

In [ ]:
# 주말 1구간(07-10시)
cluster_01_ols_fe['주말_1구간'] = 0
컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17']==1) & 
                                         (cluster_01_ols_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_01_ols_fe.loc[i,'주말_1구간'] = 1 

- 주말 2구간

In [ ]:
# 주말 2구간(10-15시)
cluster_01_ols_fe['주말_2구간'] = 0
컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17']==1) & 
                                             (cluster_01_ols_fe['컬럼13'].between(10,14))].index)
for i in 컬럼13_list:
  cluster_01_ols_fe.loc[i,'주말_2구간'] = 1

- 주말 3구간

In [ ]:
# 주말 3구간(15-18시)
cluster_01_ols_fe['주말_3구간'] = 0

컬럼13_list = list(cluster_01_ols_fe[(cluster_01_ols_fe['컬럼17']==1) & 
                                         (cluster_01_ols_fe['컬럼13'].between(15,17))].index)

for i in 컬럼13_list:
  cluster_01_ols_fe.loc[i,'주말_3구간'] = 1

In [ ]:
# 피처 제거
c_01_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_01_ols_fe.drop(c_01_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_01_ols_fe["컬럼15"]
cluster_01_ols_fe["컬럼15"] = np.log1p(cluster_01_ols_fe["컬럼15"])


cluster_01_ols_fe['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_01_ols_fe.info()

In [ ]:
target_01_ols_fe = cluster_01_ols_fe.컬럼15
data_01_ols_fe = cluster_01_ols_fe.drop(columns = '컬럼15')
data_01_ols_fe = sm.add_constant(data_01_ols_fe)

ols_model_01_fe = sm.OLS(target_01_ols_fe, data_01_ols_fe)
ols_result_01_fe = ols_model_01_fe.fit()
print(ols_result_01_fe.summary())

In [ ]:
result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_01_ols_fe.columns),
                             cluster_01_ols_fe, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
vif

### 피쳐 엔지니어링 후 성능

In [ ]:
from sklearn.metrics import mean_squared_error 

target_01_ols_fe = cluster_01_ols_fe.컬럼15
data_01_ols_fe = cluster_01_ols_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_01_ols_fe, target_01_ols_fe, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

In [ ]:
# 상위, 하위 10개 coefficients 추출
def get_top_bottom_coef(model):
    # coef series
    coef = pd.Series(model.coef_, index = data_01_ols_fe.columns)
    
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

# 회귀 계수 시각화
def visualize_coefficient(models, xlim=None):
    plt.figure(figsize=(15 , 10))
    fig.tight_layout()
    
    for i, model in enumerate(models):
        coef_high, coef_low = get_top_bottom_coef(model)
        coef_concat = pd.concat( [coef_high , coef_low], axis=0 )
        
        axs[i].set_title(model.__class__.__name__ , size=25, x= 0.9, y= 0.1)
        # axs[i].tick_params(axis="y", direction="in", pad=-120) # tick label 위치 조정
        
        # font size 조정
        for label in (axs[i].get_xticklabels() + axs[i].get_yticklabels()):
            label.set_fontsize(22)
        
        # x축 범위
        if xlim:
            axs[i].set_xlim(xlim)
        
        sns.barplot(x=coef_concat.values, y=coef_concat.index)

In [ ]:
visualize_coefficient([lr])

## 개미 Catboost Regression

In [ ]:
cluster_01_cat_fe = cluster_01_cat.copy()

In [ ]:
# 평일 1구간(7-10시)
cluster_01_cat_fe['평일_1구간'] = 0

컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17'] == 0) & 
                                         (cluster_01_cat_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'평일_1구간'] = 1

# 평일 4구간(17-20시)
cluster_01_cat_fe['평일_4구간'] = 0

컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17']==0) & 
                                         (cluster_01_cat_fe['컬럼13'].between(17,19))].index)
for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'평일_4구간'] = 1

# 평일 5구간(20-23시)
cluster_01_cat_fe['평일_5구간'] = 0

컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17']==0) & 
                                         (cluster_01_cat_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'평일_5구간'] = 1

# 주말 3구간(15-18시)
cluster_01_cat_fe['주말_3구간'] = 0

컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17']==1) & 
                                         (cluster_01_cat_fe['컬럼13'].between(15,17))].index)

for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'주말_3구간'] = 1

# 주말 1구간(07-10시)
cluster_01_cat_fe['주말_1구간'] = 0
컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17']==1) & 
                                         (cluster_01_cat_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'주말_1구간'] = 1 

# 주말 2구간(10-15시)
cluster_01_cat_fe['주말_2구간'] = 0
컬럼13_list = list(cluster_01_cat_fe[(cluster_01_cat_fe['컬럼17']==1) & 
                                             (cluster_01_cat_fe['컬럼13'].between(10,14))].index)
for i in 컬럼13_list:
  cluster_01_cat_fe.loc[i,'주말_2구간'] = 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['컬럼6']
for col in cols:
    le = LabelEncoder()
    cluster_01_cat_fe[col] = le.fit_transform(cluster_01_cat_fe[col])

In [ ]:
drop_cols_01_cat = ['컬럼3', '컬럼4', 'Cluster_num',
                    '컬럼14', '컬럼11', '컬럼13', '컬럼12', '컬럼7', '컬럼16', '컬럼8', '컬럼9', '컬럼15_G',
                    '컬럼13_bin', '컬럼15_bin', '컬럼10', '컬럼17', '컬럼2']

cluster_01_cat_fe.drop(drop_cols_01_cat, axis=1, inplace=True)

In [ ]:
target_01_cat = cluster_01_cat_fe['컬럼15']
data_df_01_cat = cluster_01_cat_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_df_01_cat, target_01_cat, test_size =0.3, random_state = 126)

In [ ]:
cluster_01_cat_fe.info()

In [ ]:
cat_01 = CatBoostRegressor(random_state=126, 
                           learning_rate = 0.01, max_depth = 8, n_estimators = 800, min_child_samples= 8)
evals = [(x_test, y_test)]

In [ ]:
cat_01.fit(x_train, y_train,  verbose=False)
pred_cat_01 = cat_01.predict(x_test)

MSE_cat_01 = mean_squared_error(y_test, pred_cat_01) 
np.sqrt(MSE_cat_01)

In [ ]:
r2_score(y_test, pred_cat_01)

In [ ]:
adj_r2_score(y_test, pred_cat_01)

In [ ]:
def plot_features_importances(model, features):
  n_features = features.shape[1]
  plt.figure(figsize=(10, 8))
  plt.barh(np.arange(n_features), sorted(model.feature_importances_), align = 'center')
  plt.yticks(np.arange(n_features), features.columns)
  plt.xlabel('Catboost Cluster01 Feature importances')
  plt.ylabel('Features')
  plt.ylim(-1, n_features)

In [ ]:
plot_features_importances(cat_01, data_df_01_cat)

# 개미와 베짱이 군집 - 베짱이 Modeling

In [ ]:
cluster_02_df = cluster_02.copy()

## feature 그룹화

In [ ]:
컬럼7_list = list(cluster_02_df[(cluster_02_df['컬럼7'] == 'CI') ].index)
for i in 컬럼7_list:
  cluster_02_df.loc[i,'컬럼7그룹'] = 4

컬럼7_list = list(cluster_02_df[(cluster_02_df['컬럼7'] == 'CJ') | (cluster_02_df['컬럼7'] == 'CE')].index)
for i in 컬럼7_list:
  cluster_02_df.loc[i,'컬럼7그룹'] = 3

컬럼7_list = list(cluster_02_df[ (cluster_02_df['컬럼7'] == 'CC')].index)
for i in 컬럼7_list:
  cluster_02_df.loc[i,'컬럼7그룹'] = 2

컬럼7_list = list(cluster_02_df[ (cluster_02_df['컬럼7'] == 'CF')].index)
for i in 컬럼7_list:
  cluster_02_df.loc[i,'컬럼7그룹'] = 1

In [ ]:
cluster_02_df.컬럼7그룹.value_counts()

In [ ]:
컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] == 12) ].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 6

컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] == 5) ].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 5

컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] == 1)].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 4

컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] == 6) | (cluster_02_df['컬럼10'] == 7)].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 3

컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] ==2) | (cluster_02_df['컬럼10'] == 8) | (cluster_02_df['컬럼10'] == 10) | (cluster_02_df['컬럼10'] == 3)].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 2

컬럼10_list = list(cluster_02_df[(cluster_02_df['컬럼10'] == 9) | (cluster_02_df['컬럼10'] == 4) | (cluster_02_df['컬럼10'] == 11)].index)
for i in 컬럼10_list:
  cluster_02_df.loc[i,'컬럼10_'] = 1

In [ ]:
cluster_02_df.컬럼10_.value_counts()

In [ ]:
컬럼12_list = list(cluster_02_df[(cluster_02_df['컬럼12'] == 5)].index)
for i in 컬럼12_list:
  cluster_02_df.loc[i,'컬럼12_'] = 4

컬럼12_list = list(cluster_02_df[(cluster_02_df['컬럼12'] == 4) ].index)
for i in 컬럼12_list:
  cluster_02_df.loc[i,'컬럼12_'] = 3

컬럼12_list = list(cluster_02_df[(cluster_02_df['컬럼12'] == 2) | (cluster_02_df['컬럼12'] == 0) |(cluster_02_df['컬럼12'] == 3) | (cluster_02_df['컬럼12'] == 6)].index)
for i in 컬럼12_list:
  cluster_02_df.loc[i,'컬럼12_'] = 2

컬럼12_list = list(cluster_02_df[(cluster_02_df['컬럼12'] == 1) ].index)
for i in 컬럼12_list:
  cluster_02_df.loc[i,'컬럼12_'] = 1

In [ ]:
cluster_02_df.컬럼12_.value_counts()

In [ ]:
컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 7) ].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 9

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 8) ].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 8

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 5)  | (cluster_02_df['컬럼13'] == 13) | (cluster_02_df['컬럼13'] == 9)  ].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 7

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 10) |  (cluster_02_df['컬럼13'] == 11)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 6

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 6) | (cluster_02_df['컬럼13'] == 12)| (cluster_02_df['컬럼13'] == 19) | (cluster_02_df['컬럼13'] == 15) | (cluster_02_df['컬럼13'] == 14)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 5

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 17) | (cluster_02_df['컬럼13'] == 20)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 4

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 18) | (cluster_02_df['컬럼13'] == 21) | (cluster_02_df['컬럼13'] == 22) | (cluster_02_df['컬럼13'] == 16)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 3

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 23)  | (cluster_02_df['컬럼13'] == 3)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 2

컬럼13_list = list(cluster_02_df[(cluster_02_df['컬럼13'] == 1) | (cluster_02_df['컬럼13'] == 2) | (cluster_02_df['컬럼13'] == 0) | (cluster_02_df['컬럼13'] == 4)].index)
for i in 컬럼13_list:
  cluster_02_df.loc[i,'컬럼13_'] = 1

In [ ]:
cluster_02_df.컬럼13_.value_counts()

## Modeling

In [ ]:
cluster_02_ols = cluster_02_df.copy()
cluster_02_cat = cluster_02_df.copy()

In [ ]:
from scipy import stats

fig, axs = plt.subplots(1,3, figsize = (15,6))

sns.histplot(cluster_02_ols["컬럼15"], ax =axs[0], kde=True)
stats.probplot(cluster_02_ols["컬럼15"], dist='norm', fit=True, plot=axs[1])
sns.histplot(np.log1p(cluster_02_ols["컬럼15"]), ax=axs[2], kde=True)

axs[0].set_title("Original 컬럼15 Histogram")
axs[2].set_title("Log Transformed 컬럼15 Histogram")

plt.show()

## 베짱이 OLS base

In [ ]:
cluster_02_ols_base = cluster_02_ols.copy()

In [ ]:
cluster_02_ols_base.info()

In [ ]:
cluster_02_ols_base.isnull().sum()

In [ ]:
# 피처 제거
c_02_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_02_ols_base.drop(c_02_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_02_ols_base["컬럼15"]
cluster_02_ols_base["컬럼15"] = np.log1p(cluster_02_ols_base["컬럼15"])


cluster_02_ols_base['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
target_02_ols_base = cluster_02_ols_base.컬럼15
data_02_ols_base = cluster_02_ols_base.drop(columns = '컬럼15')
data_02_ols_base = sm.add_constant(data_02_ols_base)

ols_model_02_base = sm.OLS(target_02_ols_base, data_02_ols_base)
ols_result_02_base = ols_model_02_base.fit()
print(ols_result_02_base.summary())

### VIF 지수 확인

In [ ]:
result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_02_ols_base.columns),
                             cluster_02_ols_base, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
print(vif)

In [ ]:
c_02_del_vif = []

for idx, row in vif.iterrows():
  if row['VIF Factor'] >= 10:
    c_02_del_vif.append(row['features'])

In [ ]:
c_02_del_vif

### 기본 성능

In [ ]:
from sklearn.metrics import mean_squared_error 

target_02_ols_base = cluster_02_ols_base.컬럼15
data_02_ols_base = cluster_02_ols_base.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_02_ols_base, target_02_ols_base, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

## 베짱이 OLS - Feature Engineering

In [ ]:
cluster_02_ols_fe = cluster_02_ols.copy()
cluster_02_ols_fe.info()

### 피쳐 추가

In [ ]:
# 평일 2구간(10-13시)
cluster_02_ols_fe['평일_2구간'] = 0
컬럼13_list = list(cluster_02_ols_fe[(cluster_02_ols_fe['컬럼17']==0) & 
                                         (cluster_02_ols_fe['컬럼13'].between(10,12))].index)
for i in 컬럼13_list:
  cluster_02_ols_fe.loc[i,'평일_2구간'] = 1


# 주말 1구간(07-10시)
cluster_02_ols_fe['주말_1구간'] = 0
컬럼13_list = list(cluster_02_ols_fe[(cluster_02_ols_fe['컬럼17']==1) & 
                                         (cluster_02_ols_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_02_ols_fe.loc[i,'주말_1구간'] = 1


# 주말 4구간(18-23시)
cluster_02_ols_fe['주말_4구간'] = 0
컬럼13_list = list(cluster_02_ols_fe[(cluster_02_ols_fe['컬럼17']==1) & 
                                         (cluster_02_ols_fe['컬럼13'].between(18,22))].index)
for i in 컬럼13_list:
  cluster_02_ols_fe.loc[i,'주말_4구간'] = 1

In [ ]:
# 피처 제거
c_02_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_02_ols_fe.drop(c_02_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_02_ols_fe["컬럼15"]
cluster_02_ols_fe["컬럼15"] = np.log1p(cluster_02_ols_fe["컬럼15"])


cluster_02_ols_fe['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_02_ols_fe.info()

In [ ]:
target_02_ols_fe = cluster_02_ols_fe.컬럼15
data_02_ols_fe = cluster_02_ols_fe.drop(columns = '컬럼15')
data_02_ols_fe = sm.add_constant(data_02_ols_fe)

ols_model_02_fe = sm.OLS(target_02_ols_fe, data_02_ols_fe)
ols_result_02_fe = ols_model_02_fe.fit()
print(ols_result_02_fe.summary())

### 피쳐 엔지니어링 후 성능

In [ ]:
target_02_ols_fe = cluster_02_ols_fe.컬럼15
data_02_ols_fe = cluster_02_ols_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_02_ols_fe, target_02_ols_fe, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
from sklearn.metrics import r2_score

r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

In [ ]:
# 상위, 하위 10개 coefficients 추출
def get_top_bottom_coef(model):
    # coef series
    coef = pd.Series(model.coef_, index = data_02_ols_fe.columns)
    
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

visualize_coefficient([lr])

## 베짱이 Catboost Regression

In [ ]:
cluster_02_cat_fe = cluster_02_cat.copy()

In [ ]:
# 평일 2구간(10-13시)
cluster_02_cat_fe['평일_2구간'] = 0
컬럼13_list = list(cluster_02_cat_fe[(cluster_02_cat_fe['컬럼17']==0) & 
                                         (cluster_02_cat_fe['컬럼13'].between(10,12))].index)
for i in 컬럼13_list:
  cluster_02_cat_fe.loc[i,'평일_2구간'] = 1


# 주말 1구간(07-10시)
cluster_02_cat_fe['주말_1구간'] = 0
컬럼13_list = list(cluster_02_cat_fe[(cluster_02_cat_fe['컬럼17']==1) & 
                                         (cluster_02_cat_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_02_cat_fe.loc[i,'주말_1구간'] = 1


# 주말 4구간(18-23시)
cluster_02_cat_fe['주말_4구간'] = 0
컬럼13_list = list(cluster_02_cat_fe[(cluster_02_cat_fe['컬럼17']==1) & 
                                         (cluster_02_cat_fe['컬럼13'].between(18,22))].index)
for i in 컬럼13_list:
  cluster_02_cat_fe.loc[i,'주말_4구간'] = 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['컬럼6']
for col in cols:
    le = LabelEncoder()
    cluster_02_cat_fe[col] = le.fit_transform(cluster_02_cat_fe[col])

In [ ]:
drop_cols_02_cat = ['컬럼3', '컬럼4', 'Cluster_num',
                    '컬럼14', '컬럼11', '컬럼13', '컬럼12', '컬럼7', '컬럼16', '컬럼8', '컬럼9', '컬럼15_G',
                    '컬럼13_bin', '컬럼15_bin', '컬럼10', '컬럼17', '컬럼2']

cluster_02_cat_fe.drop(drop_cols_02_cat, axis=1, inplace=True)

In [ ]:
target_02_cat = cluster_02_cat_fe['컬럼15']
data_df_02_cat = cluster_02_cat_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_df_02_cat, target_02_cat, test_size =0.3, random_state = 126)

In [ ]:
cat_02 = CatBoostRegressor(random_state=126, 
                           learning_rate = 0.01, max_depth = 4, n_estimators = 400, min_child_samples= 8)
evals = [(x_test, y_test)]

In [ ]:
cat_02.fit(x_train, y_train,  verbose=False)
pred_cat_02 = cat_02.predict(x_test)

MSE_cat_02 = mean_squared_error(y_test, pred_cat_02) 
np.sqrt(MSE_cat_02)

In [ ]:
r2_score(y_test, pred_cat_02)

In [ ]:
adj_r2_score(y_test, pred_cat_02)

In [ ]:
plot_features_importances(cat_02, data_df_02_cat)

# 달팽이 군집 Modeling


In [ ]:
cluster_1_df = cluster_1.copy()

## feature 그룹화

In [ ]:
컬럼7_list = list(cluster_1_df[(cluster_1_df['컬럼7'] == 'CA') | (cluster_1_df['컬럼7'] == 'CL')].index)
for i in 컬럼7_list:
  cluster_1_df.loc[i,'컬럼7그룹'] = 5

컬럼7_list = list(cluster_1_df[ (cluster_1_df['컬럼7'] == 'CK')].index)
for i in 컬럼7_list:
  cluster_1_df.loc[i,'컬럼7그룹'] = 4

컬럼7_list = list(cluster_1_df[(cluster_1_df['컬럼7'] == 'CF') | (cluster_1_df['컬럼7'] == 'CG') | (cluster_1_df['컬럼7'] == 'CD') ].index)
for i in 컬럼7_list:
  cluster_1_df.loc[i,'컬럼7그룹'] = 3

컬럼7_list = list(cluster_1_df[(cluster_1_df['컬럼7'] == 'CI') | (cluster_1_df['컬럼7'] == 'CC')].index)
for i in 컬럼7_list:
  cluster_1_df.loc[i,'컬럼7그룹'] = 2

컬럼7_list = list(cluster_1_df[(cluster_1_df['컬럼7'] == 'CJ') | (cluster_1_df['컬럼7'] == 'CE')].index)
for i in 컬럼7_list:
  cluster_1_df.loc[i,'컬럼7그룹'] = 1

In [ ]:
컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] == 12) ].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 6

컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] == 7) | (cluster_1_df['컬럼10'] == 8)].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 5

컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] == 9) | (cluster_1_df['컬럼10'] == 6) | (cluster_1_df['컬럼10'] == 10)| (cluster_1_df['컬럼10'] == 11)].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 4

컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] == 4) | (cluster_1_df['컬럼10'] == 5)].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 3

컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] ==2) | (cluster_1_df['컬럼10'] == 3)].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 2

컬럼10_list = list(cluster_1_df[(cluster_1_df['컬럼10'] == 1)].index)
for i in 컬럼10_list:
  cluster_1_df.loc[i,'컬럼10_'] = 1

In [ ]:
컬럼12_list = list(cluster_1_df[(cluster_1_df['컬럼12'] == 5)].index)
for i in 컬럼12_list:
  cluster_1_df.loc[i,'컬럼12_'] = 4

컬럼12_list = list(cluster_1_df[(cluster_1_df['컬럼12'] == 4) ].index)
for i in 컬럼12_list:
  cluster_1_df.loc[i,'컬럼12_'] = 3

컬럼12_list = list(cluster_1_df[(cluster_1_df['컬럼12'] == 2) | (cluster_1_df['컬럼12'] == 0) |(cluster_1_df['컬럼12'] == 3) | (cluster_1_df['컬럼12'] == 1)].index)
for i in 컬럼12_list:
  cluster_1_df.loc[i,'컬럼12_'] = 2

컬럼12_list = list(cluster_1_df[(cluster_1_df['컬럼12'] == 6) ].index)
for i in 컬럼12_list:
  cluster_1_df.loc[i,'컬럼12_'] = 1

In [ ]:
컬럼2_list = list(cluster_1_df[(cluster_1_df['컬럼2'] == 'ABB')].index)
for i in 컬럼2_list:
  cluster_1_df.loc[i,'컬럼2분류'] = 1

컬럼2_list = list(cluster_1_df[(cluster_1_df['컬럼2'] == 'ABE')].index)
for i in 컬럼2_list:
  cluster_1_df.loc[i,'컬럼2분류'] = 2

컬럼2_list = list(cluster_1_df[(cluster_1_df['컬럼2'] == 'ABA')].index)
for i in 컬럼2_list:
  cluster_1_df.loc[i,'컬럼2분류'] = 3

컬럼2_list = list(cluster_1_df[(cluster_1_df['컬럼2'] == 'ABD')].index)
for i in 컬럼2_list:
  cluster_1_df.loc[i,'컬럼2분류'] = 4

컬럼2_list = list(cluster_1_df[(cluster_1_df['컬럼2'] == 'ABC')].index)
for i in 컬럼2_list:
  cluster_1_df.loc[i,'컬럼2분류'] = 5

In [ ]:
컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 6) | (cluster_1_df['컬럼13'] == 7) ].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 9

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 5)  | (cluster_1_df['컬럼13'] == 8) | (cluster_1_df['컬럼13'] == 10) | (cluster_1_df['컬럼13'] == 9) ].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 8

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 12) | (cluster_1_df['컬럼13'] == 11)  ].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 7

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 18) | (cluster_1_df['컬럼13'] == 13)].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 6

컬럼13_list = list(cluster_1_df[ (cluster_1_df['컬럼13'] == 19) | (cluster_1_df['컬럼13'] == 15) | (cluster_1_df['컬럼13'] == 14)].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 5

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 17) | (cluster_1_df['컬럼13'] == 20)| (cluster_1_df['컬럼13'] == 16)].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 4

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 21) | (cluster_1_df['컬럼13'] == 4) ].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 3

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 23)  | (cluster_1_df['컬럼13'] == 22)].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 2

컬럼13_list = list(cluster_1_df[(cluster_1_df['컬럼13'] == 1) | (cluster_1_df['컬럼13'] == 2) | (cluster_1_df['컬럼13'] == 0) | (cluster_1_df['컬럼13'] == 3)].index)
for i in 컬럼13_list:
  cluster_1_df.loc[i,'컬럼13_'] = 1

## Modeling

In [ ]:
cluster_1_ols = cluster_1_df.copy()
cluster_1_cat = cluster_1_df.copy()

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (15,6))

sns.histplot(cluster_1_ols["컬럼15"], ax =axs[0], kde=True)
stats.probplot(cluster_1_ols["컬럼15"], dist='norm', fit=True, plot=axs[1])
sns.histplot(np.log1p(cluster_1_ols["컬럼15"]), ax=axs[2], kde=True)

axs[0].set_title("Original 컬럼15 Histogram")
axs[2].set_title("Log Transformed 컬럼15 Histogram")

plt.show()

## 달팽이 OLS base

In [ ]:
cluster_1_ols_base = cluster_1_ols.copy()

In [ ]:
cluster_1_ols_base.info()

In [ ]:
# 피처 제거
c_1_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_1_ols_base.drop(c_1_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_1_ols_base["컬럼15"]
cluster_1_ols_base["컬럼15"] = np.log1p(cluster_1_ols_base["컬럼15"])


cluster_1_ols_base['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
target_1_ols_base = cluster_1_ols_base.컬럼15
data_1_ols_base = cluster_1_ols_base.drop(columns = '컬럼15')
data_1_ols_base = sm.add_constant(data_1_ols_base)

ols_model_1_base = sm.OLS(target_1_ols_base, data_1_ols_base)
ols_result_1_base = ols_model_1_base.fit()
print(ols_result_1_base.summary())

### VIF 지수 확인

In [ ]:
result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_1_ols_base.columns),
                             cluster_1_ols_base, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
print(vif)

In [ ]:
c_1_lr_del_vif = []

for idx, row in vif.iterrows():
  if row['VIF Factor'] >= 10:
    c_1_lr_del_vif.append(row['features'])

In [ ]:
c_1_lr_del_vif

### 기본 성능

In [ ]:
target_1_ols_base = cluster_1_ols_base.컬럼15
data_1_ols_base = cluster_1_ols_base.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_1_ols_base, target_1_ols_base, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

## 달팽이 OLS - Feature Engineering

In [ ]:
cluster_1_ols_fe = cluster_1_ols.copy()
cluster_1_ols_fe.info()

### 피쳐 추가

In [ ]:
# 평일 1구간(7-10시)
cluster_1_ols_fe['평일_1구간'] = 0

컬럼13_list = list(cluster_1_ols_fe[(cluster_1_ols_fe['컬럼17'] == 0) & 
                                        (cluster_1_ols_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_1_ols_fe.loc[i,'평일_1구간'] = 1

# 평일 5구간(20-23시)
cluster_1_ols_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_1_ols_fe[(cluster_1_ols_fe['컬럼17']==0) & 
                                        (cluster_1_ols_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_1_ols_fe.loc[i,'평일_5구간'] = 1

In [ ]:
# 피처 제거
c_1_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_1_ols_fe.drop(c_1_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_1_ols_fe["컬럼15"]
cluster_1_ols_fe["컬럼15"] = np.log1p(cluster_1_ols_fe["컬럼15"])


cluster_1_ols_fe['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_1_ols_fe.info()

In [ ]:
target_1_ols_fe = cluster_1_ols_fe.컬럼15
data_1_ols_fe = cluster_1_ols_fe.drop(columns = '컬럼15')
data_1_ols_fe = sm.add_constant(data_1_ols_fe)

ols_model_1_fe = sm.OLS(target_1_ols_fe, data_1_ols_fe)
ols_result_1_fe = ols_model_1_fe.fit()
print(ols_result_1_fe.summary())

### 피쳐 엔지니어링 후 성능

In [ ]:
target_1_ols_fe = cluster_1_ols_fe.컬럼15
data_1_ols_fe = cluster_1_ols_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_1_ols_fe, target_1_ols_fe, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

In [ ]:
# 상위, 하위 10개 coefficients 추출
def get_top_bottom_coef(model):
    # coef series
    coef = pd.Series(model.coef_, index = data_1_ols_fe.columns)
    
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

visualize_coefficient([lr])

## 달팽이 Catboost Regression

In [ ]:
cluster_1_cat_fe = cluster_1_cat.copy()

In [ ]:
# 평일 1구간(7-10시)
cluster_1_cat_fe['평일_1구간'] = 0

컬럼13_list = list(cluster_1_cat_fe[(cluster_1_cat_fe['컬럼17'] == 0) & 
                                        (cluster_1_cat_fe['컬럼13'].between(7,9))].index)
for i in 컬럼13_list:
  cluster_1_cat_fe.loc[i,'평일_1구간'] = 1

# 평일 5구간(20-23시)
cluster_1_cat_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_1_cat_fe[(cluster_1_cat_fe['컬럼17']==0) & 
                                        (cluster_1_cat_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_1_cat_fe.loc[i,'평일_5구간'] = 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['컬럼6']
for col in cols:
    le = LabelEncoder()
    cluster_1_cat_fe[col] = le.fit_transform(cluster_1_cat_fe[col])

In [ ]:
drop_cols_1_cat = ['컬럼3', '컬럼4', 'Cluster_num',
                    '컬럼14', '컬럼11', '컬럼13', '컬럼12', '컬럼7', '컬럼16', '컬럼8', '컬럼9', '컬럼15_G',
                    '컬럼13_bin', '컬럼15_bin', '컬럼10', '컬럼17', '컬럼2']

cluster_1_cat_fe.drop(drop_cols_1_cat, axis=1, inplace=True)

In [ ]:
target_1_cat = cluster_1_cat_fe['컬럼15']
data_df_1_cat = cluster_1_cat_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_df_1_cat, target_1_cat, test_size =0.3, random_state = 126)

In [ ]:
cat_1 = CatBoostRegressor(random_state=126, 
                          learning_rate = 0.01, max_depth = 8, n_estimators = 600, min_child_samples= 8)
evals = [(x_test, y_test)]

In [ ]:
cat_1.fit(x_train, y_train,  verbose=False)
pred_cat_1 = cat_1.predict(x_test)

MSE_cat_1 = mean_squared_error(y_test, pred_cat_1) 
np.sqrt(MSE_cat_1)

In [ ]:
r2_score(y_test, pred_cat_1)

In [ ]:
adj_r2_score(y_test, pred_cat_1)

In [ ]:
plot_features_importances(cat_1, data_df_1_cat)

# 꿀벌 군집 Modeling


In [ ]:
cluster_2_df = cluster_2.copy()

## feature 그룹화

In [ ]:
컬럼7_list = list(cluster_2_df[ (cluster_2_df['컬럼7'] == 'CL')].index)
for i in 컬럼7_list:
  cluster_2_df.loc[i,'컬럼7그룹'] = 5

컬럼7_list = list(cluster_2_df[(cluster_2_df['컬럼7'] == 'CA') | (cluster_2_df['컬럼7'] == 'CD')].index)
for i in 컬럼7_list:
  cluster_2_df.loc[i,'컬럼7그룹'] = 4

컬럼7_list = list(cluster_2_df[(cluster_2_df['컬럼7'] == 'CF') | (cluster_2_df['컬럼7'] == 'CG') | (cluster_2_df['컬럼7'] == 'CK') ].index)
for i in 컬럼7_list:
  cluster_2_df.loc[i,'컬럼7그룹'] = 3

컬럼7_list = list(cluster_2_df[(cluster_2_df['컬럼7'] == 'CI') | (cluster_2_df['컬럼7'] == 'CC')].index)
for i in 컬럼7_list:
  cluster_2_df.loc[i,'컬럼7그룹'] = 2

컬럼7_list = list(cluster_2_df[(cluster_2_df['컬럼7'] == 'CJ') | (cluster_2_df['컬럼7'] == 'CE')].index)
for i in 컬럼7_list:
  cluster_2_df.loc[i,'컬럼7그룹'] = 1

In [ ]:
컬럼10_list = list(cluster_2_df[(cluster_2_df['컬럼10'] == 12) ].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 6

컬럼10_list = list(cluster_2_df[(cluster_2_df['컬럼10'] == 7) ].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 5

컬럼10_list = list(cluster_2_df[(cluster_2_df['컬럼10'] == 8) | (cluster_2_df['컬럼10'] == 6)].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 4

컬럼10_list = list(cluster_2_df[ (cluster_2_df['컬럼10'] == 9) | (cluster_2_df['컬럼10'] == 5)  | (cluster_2_df['컬럼10'] == 10)| (cluster_2_df['컬럼10'] == 11)].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 3

컬럼10_list = list(cluster_2_df[(cluster_2_df['컬럼10'] ==2) | (cluster_2_df['컬럼10'] == 4)].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 2

컬럼10_list = list(cluster_2_df[(cluster_2_df['컬럼10'] == 1) | (cluster_2_df['컬럼10'] == 3)].index)
for i in 컬럼10_list:
  cluster_2_df.loc[i,'컬럼10_'] = 1

In [ ]:
컬럼12_list = list(cluster_2_df[(cluster_2_df['컬럼12'] == 5)].index)
for i in 컬럼12_list:
  cluster_2_df.loc[i,'컬럼12_'] = 4

컬럼12_list = list(cluster_2_df[(cluster_2_df['컬럼12'] == 4) ].index)
for i in 컬럼12_list:
  cluster_2_df.loc[i,'컬럼12_'] = 3

컬럼12_list = list(cluster_2_df[(cluster_2_df['컬럼12'] == 2) | (cluster_2_df['컬럼12'] == 0) |(cluster_2_df['컬럼12'] == 3) | (cluster_2_df['컬럼12'] == 1)].index)
for i in 컬럼12_list:
  cluster_2_df.loc[i,'컬럼12_'] = 2

컬럼12_list = list(cluster_2_df[(cluster_2_df['컬럼12'] == 6) ].index)
for i in 컬럼12_list:
  cluster_2_df.loc[i,'컬럼12_'] = 1

In [ ]:
컬럼2_list = list(cluster_2_df[(cluster_2_df['컬럼2'] == 'ACD') | (cluster_2_df['컬럼2'] == 'ACA')].index)
for i in 컬럼2_list:
  cluster_2_df.loc[i,'컬럼2분류'] = 1

컬럼2_list = list(cluster_2_df[(cluster_2_df['컬럼2'] == 'ACB')].index)
for i in 컬럼2_list:
  cluster_2_df.loc[i,'컬럼2분류'] = 2

컬럼2_list = list(cluster_2_df[(cluster_2_df['컬럼2'] == 'ACC')].index)
for i in 컬럼2_list:
  cluster_2_df.loc[i,'컬럼2분류'] = 3

컬럼2_list = list(cluster_2_df[(cluster_2_df['컬럼2'] == 'ACE')].index)
for i in 컬럼2_list:
  cluster_2_df.loc[i,'컬럼2분류'] = 4

In [ ]:
컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 6) ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 9

컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 7) ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 8

컬럼13_list = list(cluster_2_df[ (cluster_2_df['컬럼13'] == 8) ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 7

컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 5)   | (cluster_2_df['컬럼13'] == 10) | (cluster_2_df['컬럼13'] == 9) ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 6

컬럼13_list = list(cluster_2_df[ (cluster_2_df['컬럼13'] == 12) | (cluster_2_df['컬럼13'] == 11)  ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 5

컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 18) | (cluster_2_df['컬럼13'] == 13) | (cluster_2_df['컬럼13'] == 20)| (cluster_2_df['컬럼13'] == 16)].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 4

컬럼13_list = list(cluster_2_df[ (cluster_2_df['컬럼13'] == 17) | (cluster_2_df['컬럼13'] == 4)| (cluster_2_df['컬럼13'] == 19) | (cluster_2_df['컬럼13'] == 15) | (cluster_2_df['컬럼13'] == 14)].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 3

컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 21) | (cluster_2_df['컬럼13'] == 23)  | (cluster_2_df['컬럼13'] == 22) | (cluster_2_df['컬럼13'] == 3)].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 2

컬럼13_list = list(cluster_2_df[(cluster_2_df['컬럼13'] == 1) | (cluster_2_df['컬럼13'] == 2) | (cluster_2_df['컬럼13'] == 0) ].index)
for i in 컬럼13_list:
  cluster_2_df.loc[i,'컬럼13_'] = 1

## Modeling

In [ ]:
cluster_2_ols = cluster_2_df.copy()
cluster_2_cat = cluster_2_df.copy()

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (15,6))

sns.histplot(cluster_2_ols["컬럼15"], ax =axs[0], kde=True)
stats.probplot(cluster_2_ols["컬럼15"], dist='norm', fit=True, plot=axs[1])
sns.histplot(np.log1p(cluster_2_ols["컬럼15"]), ax=axs[2], kde=True)

axs[0].set_title("Original 컬럼15 Histogram")
axs[2].set_title("Log Transformed 컬럼15 Histogram")

plt.show()

## 꿀벌 OLS base

In [ ]:
cluster_2_ols_base = cluster_2_ols.copy()
cluster_2_ols_base.info()

In [ ]:
# 피처 제거
c_2_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_2_ols_base.drop(c_2_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_2_ols_base["컬럼15"]
cluster_2_ols_base["컬럼15"] = np.log1p(cluster_2_ols_base["컬럼15"])


cluster_2_ols_base['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
target_2_ols_base = cluster_2_ols_base.컬럼15
data_2_ols_base = cluster_2_ols_base.drop(columns = '컬럼15')
data_2_ols_base = sm.add_constant(data_2_ols_base)

ols_model_2_base = sm.OLS(target_2_ols_base, data_2_ols_base)
ols_result_2_base = ols_model_2_base.fit()
print(ols_result_2_base.summary())

### VIF 지수 확인

In [ ]:
result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_2_ols_base.columns),
                             cluster_2_ols_base, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
print(vif)

In [ ]:
c_2_lr_del_vif = []

for idx, row in vif.iterrows():
  if row['VIF Factor'] >= 10:
    c_2_lr_del_vif.append(row['features'])

In [ ]:
c_2_lr_del_vif

### 기본 성능

In [ ]:
target_2_ols_base = cluster_2_ols_base.컬럼15
data_2_ols_base = cluster_2_ols_base.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_2_ols_base, target_2_ols_base, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

## 꿀벌 OLS - Feature Engineering

In [ ]:
cluster_2_ols_fe = cluster_2_ols.copy()
cluster_2_ols_fe.info()

### 피쳐추가

In [ ]:
# 평일 5구간(20-23시)
cluster_2_ols_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_2_ols_fe[(cluster_2_ols_fe['컬럼17']==0) & 
                                             (cluster_2_ols_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_2_ols_fe.loc[i,'평일_5구간'] = 1

# 주말 3구간(15-18시)
cluster_2_ols_fe['주말_3구간'] = 0
컬럼13_list = list(cluster_2_ols_fe[(cluster_2_ols_fe['컬럼17']==1) & 
                                        (cluster_2_ols_fe['컬럼13'].between(15,17))].index)
for i in 컬럼13_list:
  cluster_2_ols_fe.loc[i,'주말_3구간'] = 1

In [ ]:
# 피처 제거
c_2_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_2_ols_fe.drop(c_2_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_2_ols_fe["컬럼15"]
cluster_2_ols_fe["컬럼15"] = np.log1p(cluster_2_ols_fe["컬럼15"])


cluster_2_ols_fe['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_2_ols_fe.info()

In [ ]:
target_2_ols_fe = cluster_2_ols_fe.컬럼15
data_2_ols_fe = cluster_2_ols_fe.drop(columns = '컬럼15')
data_2_ols_fe = sm.add_constant(data_2_ols_fe)

ols_model_2_fe = sm.OLS(target_2_ols_fe, data_2_ols_fe)
ols_result_2_fe = ols_model_2_fe.fit()
print(ols_result_2_fe.summary())

### 피쳐 엔지니어링 후 성능

In [ ]:
target_2_ols_fe = cluster_2_ols_fe.컬럼15
data_2_ols_fe = cluster_2_ols_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_2_ols_fe, target_2_ols_fe, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

In [ ]:
# 상위, 하위 10개 coefficients 추출
def get_top_bottom_coef(model):
    # coef series
    coef = pd.Series(model.coef_, index = data_2_ols_fe.columns)
    
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

visualize_coefficient([lr])

## 꿀벌 Catboost Regression

In [ ]:
cluster_2_cat_fe = cluster_2_cat.copy()

In [ ]:
# 평일 5구간(20-23시)
cluster_2_cat_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_2_cat_fe[(cluster_2_cat_fe['컬럼17']==0) & 
                                             (cluster_2_cat_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_2_cat_fe.loc[i,'평일_5구간'] = 1

# 주말 3구간(15-18시)
cluster_2_cat_fe['주말_3구간'] = 0
컬럼13_list = list(cluster_2_cat_fe[(cluster_2_cat_fe['컬럼17']==1) & 
                                        (cluster_2_cat_fe['컬럼13'].between(15,17))].index)
for i in 컬럼13_list:
  cluster_2_cat_fe.loc[i,'주말_3구간'] = 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['컬럼6', '컬럼2', '컬럼7그룹']
for col in cols:
    le = LabelEncoder()
    cluster_2_cat_fe[col] = le.fit_transform(cluster_2_cat_fe[col])

In [ ]:
drop_cols_2_cat = ['컬럼3', '컬럼4', 'Cluster_num',
                    '컬럼14', '컬럼11', '컬럼13', '컬럼12', '컬럼7', '컬럼16', '컬럼8', '컬럼9', '컬럼15_G',
                    '컬럼13_bin', '컬럼15_bin', '컬럼10', '컬럼17', '컬럼2']

cluster_2_cat_fe.drop(drop_cols_2_cat, axis=1, inplace=True)

In [ ]:
target_2_cat = cluster_2_cat_fe['컬럼15']
data_df_2_cat = cluster_2_cat_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_df_2_cat, target_2_cat, test_size =0.3, random_state = 126)

In [ ]:
cat_2 = CatBoostRegressor(random_state=126, 
                          learning_rate = 0.01, max_depth = 8, n_estimators = 600, min_child_samples= 12)
evals = [(x_test, y_test)]

In [ ]:
cat_2.fit(x_train, y_train,  verbose=False)
pred_cat_2 = cat_2.predict(x_test)

MSE_cat_2 = mean_squared_error(y_test, pred_cat_2) 
np.sqrt(MSE_cat_2)

In [ ]:
r2_score(y_test, pred_cat_2)

In [ ]:
adj_r2_score(y_test, pred_cat_2)

In [ ]:
plot_features_importances(cat_2, data_df_2_cat)

# 반딧불이 군집 Modeling


In [ ]:
cluster_3_df = cluster_3.copy()

## feature 그룹화

In [ ]:
컬럼7_list = list(cluster_3_df[ (cluster_3_df['컬럼7'] == 'CL')].index)
for i in 컬럼7_list:
  cluster_3_df.loc[i,'컬럼7그룹'] = 5

컬럼7_list = list(cluster_3_df[(cluster_3_df['컬럼7'] == 'CG') | (cluster_3_df['컬럼7'] == 'CC')].index)
for i in 컬럼7_list:
  cluster_3_df.loc[i,'컬럼7그룹'] = 4

컬럼7_list = list(cluster_3_df[(cluster_3_df['컬럼7'] == 'CI') ].index)
for i in 컬럼7_list:
  cluster_3_df.loc[i,'컬럼7그룹'] = 3

컬럼7_list = list(cluster_3_df[(cluster_3_df['컬럼7'] == 'CJ') | (cluster_3_df['컬럼7'] == 'CD') |  (cluster_3_df['컬럼7'] == 'CE') |  (cluster_3_df['컬럼7'] == 'CF')].index)
for i in 컬럼7_list:
  cluster_3_df.loc[i,'컬럼7그룹'] = 2

컬럼7_list = list(cluster_3_df[(cluster_3_df['컬럼7'] == 'CK') ].index)
for i in 컬럼7_list:
  cluster_3_df.loc[i,'컬럼7그룹'] = 1

In [ ]:
컬럼10_list = list(cluster_3_df[(cluster_3_df['컬럼10'] == 12) ].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 6

컬럼10_list = list(cluster_3_df[(cluster_3_df['컬럼10'] == 7) | (cluster_3_df['컬럼10'] == 8)].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 5

컬럼10_list = list(cluster_3_df[(cluster_3_df['컬럼10'] == 6) | (cluster_3_df['컬럼10'] == 9) | (cluster_3_df['컬럼10'] == 10) | (cluster_3_df['컬럼10'] == 11)].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 4

컬럼10_list = list(cluster_3_df[ (cluster_3_df['컬럼10'] == 5) | (cluster_3_df['컬럼10'] == 4)].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 3

컬럼10_list = list(cluster_3_df[(cluster_3_df['컬럼10'] ==2) | (cluster_3_df['컬럼10'] == 3)].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 2

컬럼10_list = list(cluster_3_df[(cluster_3_df['컬럼10'] == 1) ].index)
for i in 컬럼10_list:
  cluster_3_df.loc[i,'컬럼10_'] = 1

In [ ]:
컬럼12_list = list(cluster_3_df[(cluster_3_df['컬럼12'] == 5)].index)
for i in 컬럼12_list:
  cluster_3_df.loc[i,'컬럼12_'] = 4

컬럼12_list = list(cluster_3_df[(cluster_3_df['컬럼12'] == 4) | (cluster_3_df['컬럼12'] == 1) |(cluster_3_df['컬럼12'] == 3)].index)
for i in 컬럼12_list:
  cluster_3_df.loc[i,'컬럼12_'] = 3

컬럼12_list = list(cluster_3_df[(cluster_3_df['컬럼12'] == 2) | (cluster_3_df['컬럼12'] == 0) ].index)
for i in 컬럼12_list:
  cluster_3_df.loc[i,'컬럼12_'] = 2

컬럼12_list = list(cluster_3_df[(cluster_3_df['컬럼12'] == 6) ].index)
for i in 컬럼12_list:
  cluster_3_df.loc[i,'컬럼12_'] = 1

In [ ]:
컬럼2_list = list(cluster_3_df[(cluster_3_df['컬럼2'] == 'ADA') ].index)
for i in 컬럼2_list:
  cluster_3_df.loc[i,'컬럼2분류'] = 1

컬럼2_list = list(cluster_3_df[(cluster_3_df['컬럼2'] == 'ADC')].index)
for i in 컬럼2_list:
  cluster_3_df.loc[i,'컬럼2분류'] = 2

컬럼2_list = list(cluster_3_df[(cluster_3_df['컬럼2'] == 'ADB')].index)
for i in 컬럼2_list:
  cluster_3_df.loc[i,'컬럼2분류'] = 3

In [ ]:
컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 6) ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 9

컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 7) ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 8

컬럼13_list = list(cluster_3_df[ (cluster_3_df['컬럼13'] == 5) | (cluster_3_df['컬럼13'] == 8) ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 7

컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 10) | (cluster_3_df['컬럼13'] == 9) ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 6

컬럼13_list = list(cluster_3_df[ (cluster_3_df['컬럼13'] == 12) | (cluster_3_df['컬럼13'] == 11)  ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 5

컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 18) | (cluster_3_df['컬럼13'] == 13) | (cluster_3_df['컬럼13'] == 14)| (cluster_3_df['컬럼13'] == 15)].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 4

컬럼13_list = list(cluster_3_df[ (cluster_3_df['컬럼13'] == 17) | (cluster_3_df['컬럼13'] == 20)| (cluster_3_df['컬럼13'] == 19) | (cluster_3_df['컬럼13'] == 16) | (cluster_3_df['컬럼13'] == 4)].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 3

컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 21) | (cluster_3_df['컬럼13'] == 23)  | (cluster_3_df['컬럼13'] == 22) | (cluster_3_df['컬럼13'] == 3)].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 2

컬럼13_list = list(cluster_3_df[(cluster_3_df['컬럼13'] == 1) | (cluster_3_df['컬럼13'] == 2) | (cluster_3_df['컬럼13'] == 0) ].index)
for i in 컬럼13_list:
  cluster_3_df.loc[i,'컬럼13_'] = 1

In [ ]:
cluster_3_df.isnull().sum()

## Modeling

In [ ]:
cluster_3_ols = cluster_3_df.copy()
cluster_3_cat = cluster_3_df.copy()

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (15,6))

sns.histplot(cluster_3_ols["컬럼15"], ax =axs[0], kde=True)
stats.probplot(cluster_3_ols["컬럼15"], dist='norm', fit=True, plot=axs[1])
sns.histplot(np.log1p(cluster_3_ols["컬럼15"]), ax=axs[2], kde=True)

axs[0].set_title("Original 컬럼15 Histogram")
axs[2].set_title("Log Transformed 컬럼15 Histogram")

plt.show()

## 반딧불이 OLS base

In [ ]:
cluster_3_ols_base = cluster_3_ols.copy()
cluster_3_ols_base.info()

In [ ]:
# 피처 제거
c_3_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_3_ols_base.drop(c_3_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_3_ols_base["컬럼15"]
cluster_3_ols_base["컬럼15"] = np.log1p(cluster_3_ols_base["컬럼15"])


cluster_3_ols_base['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
target_3_ols_base = cluster_3_ols_base.컬럼15
data_3_ols_base = cluster_3_ols_base.drop(columns = '컬럼15')
data_3_ols_base = sm.add_constant(data_3_ols_base)

ols_model_3_base = sm.OLS(target_3_ols_base, data_3_ols_base)
ols_result_3_base = ols_model_3_base.fit()
print(ols_result_3_base.summary())

### VIF 지수 확인

In [ ]:
result1, result2 = dmatrices('컬럼15 ~ + ' + '+'.join(cluster_3_ols_base.columns),
                             cluster_3_ols_base, return_type = 'dataframe')

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(result2.values, i) for i in range(result2.shape[1])]
vif['features'] = result2.columns
print(vif)

In [ ]:
c_3_lr_del_vif = []

for idx, row in vif.iterrows():
  if row['VIF Factor'] >= 10:
    c_3_lr_del_vif.append(row['features'])

In [ ]:
c_3_lr_del_vif

### 기본 성능

In [ ]:
target_3_ols_base = cluster_3_ols_base.컬럼15
data_3_ols_base = cluster_3_ols_base.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_3_ols_base, target_3_ols_base, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

## 반딧불이 OLS - Feature Engineering

In [ ]:
cluster_3_ols_fe = cluster_3_ols.copy()
cluster_3_ols_fe.info()

### 피쳐 추가

In [ ]:
# 평일 4구간(17-20시)
cluster_3_ols_fe['평일_4구간'] = 0
컬럼13_list = list(cluster_3_ols_fe[(cluster_3_ols_fe['컬럼17']==0) & 
                                        (cluster_3_ols_fe['컬럼13'].between(17,19))].index)
for i in 컬럼13_list:
  cluster_3_ols_fe.loc[i,'평일_4구간'] = 1

# 평일 5구간(20-23시)
cluster_3_ols_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_3_ols_fe[(cluster_3_ols_fe['컬럼17']==0) & 
                                             (cluster_3_ols_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_3_ols_fe.loc[i,'평일_5구간'] = 1

# 평일 6구간(23-02시)
cluster_3_ols_fe['평일_6구간'] = 0
컬럼13_list = list(cluster_3_ols_fe[(cluster_3_ols_fe['컬럼17']==0) & 
                                       ((cluster_3_ols_fe['컬럼13']==0) | (cluster_3_ols_fe['컬럼13']==1) | (cluster_3_ols_fe['컬럼13']==23))].index)
for i in 컬럼13_list:
  cluster_3_ols_fe.loc[i,'평일_6구간'] = 1

In [ ]:
# 피처 제거
c_3_base_drop_cols = ['컬럼16', '컬럼3', '컬럼4', '컬럼14', '컬럼11', '컬럼9', '컬럼8', 
                     '컬럼15_G', '컬럼15_bin','Cluster_num', '컬럼7', '컬럼13_bin', '컬럼13', '컬럼12', '컬럼2', '컬럼10', '컬럼17']

cluster_3_ols_fe.drop(c_3_base_drop_cols, axis=1, inplace=True)


# 종속 변수 로그 변환
org_SalePrice = cluster_3_ols_fe["컬럼15"]
cluster_3_ols_fe["컬럼15"] = np.log1p(cluster_3_ols_fe["컬럼15"])


cluster_3_ols_fe['컬럼6'].replace({'B' : 0, 'A' : 1}, inplace=True) 

In [ ]:
cluster_3_ols_fe.info()

In [ ]:
target_3_ols_fe = cluster_3_ols_fe.컬럼15
data_3_ols_fe = cluster_3_ols_fe.drop(columns = '컬럼15')
data_3_ols_fe = sm.add_constant(data_3_ols_fe)

ols_model_3_fe = sm.OLS(target_3_ols_fe, data_3_ols_fe)
ols_result_3_fe = ols_model_3_fe.fit()
print(ols_result_3_fe.summary())

### 피쳐 엔지니어링 후 성능

In [ ]:
target_3_ols_fe = cluster_3_ols_fe.컬럼15
data_3_ols_fe = cluster_3_ols_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_3_ols_fe, target_3_ols_fe, test_size =0.3, random_state = 126)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred_lr = lr.predict(x_test)
MSE_lr = mean_squared_error(y_test, pred_lr)
np.sqrt(MSE_lr) 

In [ ]:
r2_score(y_test, pred_lr)

In [ ]:
adj_r2_score(y_test, pred_lr)

In [ ]:
# 상위, 하위 10개 coefficients 추출
def get_top_bottom_coef(model):
    # coef series
    coef = pd.Series(model.coef_, index = data_3_ols_fe.columns)
    
    coef_high = coef.sort_values(ascending=False).head(10)
    coef_low = coef.sort_values(ascending=False).tail(10)
    return coef_high, coef_low

visualize_coefficient([lr])

## 반딧불이 Catboost Regression

In [ ]:
cluster_3_cat_fe = cluster_3_cat.copy()

In [ ]:
# 평일 4구간(17-20시)
cluster_3_cat_fe['평일_4구간'] = 0
컬럼13_list = list(cluster_3_cat_fe[(cluster_3_cat_fe['컬럼17']==0) & 
                                        (cluster_3_cat_fe['컬럼13'].between(17,19))].index)
for i in 컬럼13_list:
  cluster_3_cat_fe.loc[i,'평일_4구간'] = 1

# 평일 5구간(20-23시)
cluster_3_cat_fe['평일_5구간'] = 0
컬럼13_list = list(cluster_3_cat_fe[(cluster_3_cat_fe['컬럼17']==0) & 
                                             (cluster_3_cat_fe['컬럼13'].between(20,22))].index)
for i in 컬럼13_list:
  cluster_3_cat_fe.loc[i,'평일_5구간'] = 1

# 평일 6구간(23-02시)
cluster_3_cat_fe['평일_6구간'] = 0
컬럼13_list = list(cluster_3_cat_fe[(cluster_3_cat_fe['컬럼17']==0) & 
                                       ((cluster_3_cat_fe['컬럼13']==0) | (cluster_3_cat_fe['컬럼13']==1) | (cluster_3_cat_fe['컬럼13']==23))].index)
for i in 컬럼13_list:
  cluster_3_cat_fe.loc[i,'평일_6구간'] = 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['컬럼6']
for col in cols:
    le = LabelEncoder()
    cluster_3_cat_fe[col] = le.fit_transform(cluster_3_cat_fe[col])

In [ ]:
drop_cols_3_cat = ['컬럼3', '컬럼4', 'Cluster_num',
                    '컬럼14', '컬럼11', '컬럼13', '컬럼12', '컬럼7', '컬럼16', '컬럼8', '컬럼9', '컬럼15_G',
                    '컬럼13_bin', '컬럼15_bin', '컬럼10', '컬럼17', '컬럼2']

cluster_3_cat_fe.drop(drop_cols_3_cat, axis=1, inplace=True)

In [ ]:
target_3_cat = cluster_3_cat_fe['컬럼15']
data_df_3_cat = cluster_3_cat_fe.drop(columns = '컬럼15')
x_train, x_test, y_train, y_test = train_test_split(data_df_3_cat, target_3_cat, test_size =0.3, random_state = 126)

In [ ]:
cat_3 = CatBoostRegressor(random_state=126, 
                          learning_rate = 0.01, max_depth = 8, n_estimators = 400, min_child_samples= 6)
evals = [(x_test, y_test)]

In [ ]:
cat_3.fit(x_train, y_train,  verbose=False)
pred_cat_3 = cat_3.predict(x_test)

MSE_cat_3 = mean_squared_error(y_test, pred_cat_3) 
np.sqrt(MSE_cat_3)

In [ ]:
r2_score(y_test, pred_cat_3)

In [ ]:
adj_r2_score(y_test, pred_cat_3)

In [ ]:
plot_features_importances(cat_3, data_df_3_cat)